Início

In [2]:
import pandas as pd
import requests
import os
import zipfile

In [3]:
diretorio_atual = os.getcwd()
os.chdir(f"{diretorio_atual}/dados_cvm")

Importação de bases

In [4]:
anos = range(2010,2023)
url_base = 'https://dados.cvm.gov.br/dados/cia_aberta/doc/DFP/DADOS/'

for ano in anos:
    download = requests.get(url_base+f'dfp_cia_aberta_{ano}.zip')

    open(f'dfp_cia_aberta_{ano}.zip', 'wb').write(download.content)

print(pd.Timestamp.now())

2022-09-27 22:28:45.671692


Construção e formatação da base de dados

In [5]:
pd.Timestamp.now()
lista_demonstracoes = []
diretorio_atual = os.getcwd()
for arquivo in os.listdir(diretorio_atual):
    if arquivo == '.ipynb_checkpoints':
        pass
    else:
        arquivo_zip = zipfile.ZipFile(arquivo)

        for planilha in arquivo_zip.namelist():
            demonstracao = pd.read_csv(arquivo_zip.open(planilha),sep=';', encoding='ISO-8859-1',
                                        dtype={'ORDEM_EXERC':'category'})
            lista_demonstracoes.append(demonstracao)
pd.Timestamp.now()

Timestamp('2022-09-27 22:29:27.283599')

In [6]:
base_dados = pd.concat(lista_demonstracoes)
base_dados[:5]
pd.Timestamp.now()

Timestamp('2022-09-27 22:29:33.170562')

In [8]:
base_dados[['con_ind','tipo_dem']] = base_dados['GRUPO_DFP'].str.split('-',expand=True)
pd.Timestamp.now()

Timestamp('2022-09-27 22:32:36.370923')

In [9]:
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
pd.Timestamp.now()

Timestamp('2022-09-27 22:32:41.147278')

In [10]:
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()
pd.Timestamp.now()

Timestamp('2022-09-27 22:32:46.755277')

In [11]:
base_dados = base_dados[base_dados['ORDEM_EXERC'] != 'PENÚLTIMO']
pd.Timestamp.now()

Timestamp('2022-09-27 22:34:37.695405')

In [12]:
lista_dem = base_dados['tipo_dem'].unique()
#base_dados['tipo_dem'].unique()
print(lista_dem)
pd.Timestamp.now()

[nan 'Balanço Patrimonial Ativo' 'Balanço Patrimonial Passivo'
 'Demonstração do Fluxo de Caixa (Método Direto)'
 'Demonstração do Fluxo de Caixa (Método Indireto)'
 'Demonstração das Mutações do Patrimônio Líquido'
 'Demonstração de Resultado Abrangente' 'Demonstração do Resultado'
 'Demonstração de Valor Adicionado']


Timestamp('2022-09-27 22:34:39.429674')

In [13]:
lista_empresas = base_dados['DENOM_CIA'].unique()
print(lista_empresas)
pd.Timestamp.now()

['BCO BRASIL S.A.' 'BRB BCO DE BRASILIA S.A.'
 'CENTRAIS ELET BRAS S.A. - ELETROBRAS' ...
 'UNIÃO QUÍMICA FARMACÊUTICA NACIONAL S.A.' 'CLARANET TECHNOLOGY S.A.'
 'SENIOR SISTEMAS S.A.']


Timestamp('2022-09-27 22:34:40.113941')

In [14]:
lista_conta = base_dados['DS_CONTA'].unique()
print(lista_conta)
pd.Timestamp.now()

[nan 'Ativo Total' 'Caixa e Equivalentes de Caixa' ...
 'Demais despesas financeiras'
 'Ganho líquido decorrente de mudança no valor justo e realização da mais ou menos valia dos ativos bi'
 '41689']


Timestamp('2022-09-27 22:34:40.627990')

Filtro para empresas desejadas

In [22]:
filtro = ['BCO BRASIL S.A.', 'BRB BCO DE BRASILIA S.A.','IRB - BRASIL RESSEGUROS S.A.']
filtro_dre = base_dados[base_dados.DENOM_CIA.isin(filtro)
                        #&(base_dados['tipo_dem']=='Demonstração de Resultados')
                        ]

In [23]:
lista_conta = filtro_dre['DS_CONTA'].unique()
lista_dem = filtro_dre['tipo_dem'].unique()


In [24]:
campos = ['Imposto de Renda e Contribuição Social sobre o Lucro'
            ,'Receitas das Operações'
            ,'Lucro Líquido do Período'
            ,'Lucro Líquido Consolidado do Período'
            ,'Lucro/Prejuízo Consolidado do Período'
            ,'Lucro/Prejuízo do Período'
            ,'Imposto de Renda e Contribuição Social sobre o Lucro'
            ,'Depreciação, Amortização e Exaustão']
filtro_dre2 = filtro_dre[filtro_dre.DS_CONTA.isin(campos)]

In [25]:
filtro_dre2

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,...,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF,con_ind,tipo_dem
257,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração das Mutações do ...,...,ÚLTIMO,2010-12-31,5.05.01,Lucro Líquido do Período,0.0,S,2010-01-01,Capital Social Integralizado,DF Consolidado,Demonstração das Mutações do Patrimônio Líquido
259,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração das Mutações do ...,...,ÚLTIMO,2010-12-31,5.05.01,Lucro Líquido do Período,0.0,S,2010-01-01,"Reservas de Capital, Opções Outorgadas e Ações...",DF Consolidado,Demonstração das Mutações do Patrimônio Líquido
261,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração das Mutações do ...,...,ÚLTIMO,2010-12-31,5.05.01,Lucro Líquido do Período,0.0,S,2010-01-01,Reservas de Lucro,DF Consolidado,Demonstração das Mutações do Patrimônio Líquido
263,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração das Mutações do ...,...,ÚLTIMO,2010-12-31,5.05.01,Lucro Líquido do Período,11296009.0,S,2010-01-01,Lucros ou Prejuízos Acumulados,DF Consolidado,Demonstração das Mutações do Patrimônio Líquido
265,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração das Mutações do ...,...,ÚLTIMO,2010-12-31,5.05.01,Lucro Líquido do Período,0.0,S,2010-01-01,Outros Resultados Abrangentes,DF Consolidado,Demonstração das Mutações do Patrimônio Líquido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,00.000.208/0001-00,2021-12-31,3,BRB BCO DE BRASILIA S.A.,14206,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração de Valor Adicionado,...,ÚLTIMO,2021-12-31,7.05.01,"Depreciação, Amortização e Exaustão",-131565.0,S,2021-12-01,NaN,DF Consolidado,Demonstração de Valor Adicionado
24715,33.376.989/0001-91,2021-12-31,1,IRB - BRASIL RESSEGUROS S.A.,24180,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração de Valor Adicionado,...,ÚLTIMO,2021-12-31,7.07.01,"Depreciação, Amortização e Exaustão",-44374.0,S,2021-01-01,NaN,DF Consolidado,Demonstração de Valor Adicionado
45,00.000.000/0001-91,2021-12-31,2,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,ÚLTIMO,2021-12-31,7.05.01,"Depreciação, Amortização e Exaustão",-2763722.0,S,2021-01-01,NaN,DF Individual,Demonstração de Valor Adicionado
125,00.000.208/0001-00,2021-12-31,3,BRB BCO DE BRASILIA S.A.,14206,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,ÚLTIMO,2021-12-31,7.05.01,"Depreciação, Amortização e Exaustão",-98184.0,S,2021-12-01,NaN,DF Individual,Demonstração de Valor Adicionado
